<a href="https://colab.research.google.com/github/stevencao123/movie_recommender/blob/main/Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import pandas as pd
import numpy as np
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [53]:
from google.colab import files
uploaded = files.upload()

Saving movie_dat.csv to movie_dat (1).csv


In [56]:
# printing df to check for necessary features
df = pd.read_csv('movie_dat.csv')
df.head(2)

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0


In [119]:
# extracting keywords from description
nltk.download('punkt')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

keywords = df['Description']
keywords = keywords.str.lower().str.replace('\W', ' ')
keywords = keywords.apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))
df['Keywords'] = keywords
df.head(3)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,features,Keywords
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0,Guardians of the Galaxy Action Adventure Sci-F...,group intergalactic criminals forced work toge...
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0,Prometheus Adventure Mystery Sci-Fi Following ...,following clues origin mankind team finds stru...
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0,Split Horror Thriller Three girls are kidnappe...,three girls kidnapped man diagnosed 23 distinc...


In [63]:
# defining features
col = ['Title', 'Genre', 'Description', 'Director', 'Actors', ]
df[col].head(3)

,Title,Genre,Description,Director,Actors
0,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S..."
1,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa..."
2,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar..."


In [64]:
# creating index list for titles
index = pd.Series(df.index, index=df['Title'])
index.head(3)

Title
Guardians of the Galaxy    0
Prometheus                 1
Split                      2
dtype: int64

In [65]:
# combining features and removing commas for vectorizer
features = df[df.columns[1:5]].apply(
    lambda x: ' '.join(x),
    axis = 1
)
features = features.replace(',', ' ', regex=True)
df['features'] = features

In [59]:
# fitting and transforming features
count = CountVectorizer(stop_words='english')
cm = count.fit_transform(df['features'])
cm.shape

(1000, 6995)

In [60]:
# computing the cosine similarity of the features
cs = cosine_similarity(tfidf_matrix, tfidf_matrix)
cs.shape

(1000, 1000)

In [125]:
def get_rec(title, cs=cs):
  i = index[title]
  s_score = list(enumerate(cs[i]))
  s_score = sorted(s_score, key=lambda x: x[1], reverse = True)
  s_score = s_score[1:11]
  movie_rec = [i[0] for i in s_score]
  return df['Title'].iloc[movie_rec]

In [128]:
get_rec('Doctor Strange')

158            Scott Pilgrim vs. the World
47                                  Fallen
821    The Imaginarium of Doctor Parnassus
562                               Sinister
344                           Spider-Man 3
33                                Deadpool
771                         The Assignment
619        The Secret Life of Walter Mitty
865                                  Horns
648                        Sleeping Beauty
Name: Title, dtype: object